In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc
import os, warnings 
warnings.filterwarnings('ignore') 

import stan

In [2]:
path = Path('outputs_image_weight')
if not os.path.exists(path):
    os.makedirs(path)

To analyze the impact of the weight multiplied to the image features.

In [3]:
def stan_wrap(adata):
    adata = stan.add_gene_tf_matrix(adata, min_cells_proportion = 0.2, min_tfs_per_gene= 5, min_genes_per_tf= 10,
                                    gene_tf_source="hTFtarget", tf_list="humantfs", source_dir="resources/")
    stan.pixel_intensity(adata, windowsize=25)
    
    sc.pp.normalize_total(adata)
    adata.layers['scaled'] = np.sqrt(adata.to_df())
    stan.assign_folds(adata, n_folds=10, random_seed=0)
    
    for iw in np.arange(0, 1.1, 0.1):
        print(f'Image weight: {iw}')
        if iw==0:
            stan.make_kernel(adata, X=adata.obsm['spatial'], n=250, kernel_name=f'kernel_{iw}')
        else:
            stan.make_kernel_from_pixel(adata, n=250, im_feats_weight=iw, kernel_name=f'kernel_{iw}')
    
        stan_model = stan.Stan(adata, layer='scaled', kernel_name=f'kernel_{iw}')
        stan_model.fit(n_steps=3, stages=1,
                       grid_search_params={'lam1':[1e-2, 1e2], 'lam2':[1e-2, 1e2]})
        print(stan_model.params)

        cor, gene_cor = stan_model.evaluate(fold=-1)
        adata.obs[f'pred_cor_stan_{iw}'] = cor
        print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))

        adata.obsm[f'tfa_stan_{iw}'] = pd.DataFrame(
            stan_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])
    return adata

### Lymphnode

In [4]:
sample_id = "V1_Human_Lymph_Node"
adata = stan.read_visium_sge(sample_id=sample_id, min_cells=5, min_counts=5000)
adata = stan_wrap(adata)
adata.write(path / ('adata_'+sample_id+'.h5ad'))

Image weight: 0.0
Time elapsed: 31.23 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2242
Image weight: 0.1
Time elapsed: 32.26 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.224
Image weight: 0.2
Time elapsed: 32.06 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.224
Image weight: 0.30000000000000004
Time elapsed: 32.26 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.224
Image weight: 0.4
Time elapsed: 32.10 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.224
Image weight: 0.5
Time elapsed: 32.25 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2239
Image weight: 0.6000000000000001
Time elapsed: 32.04 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2237
Image weight: 0.7000000000000001
Time elapsed: 32.05 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2234
Image weight: 0.8
Time elapsed: 32.29 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2233
Imag

### Tonsil

In [12]:
sample_list = ["CytAssist_FFPE_Protein_Expression_Human_Tonsil",
               "CytAssist_FFPE_Protein_Expression_Human_Tonsil_AddOns"]
for sample_id in sample_list:
    adata, pdata = stan.read_cytassist(sample_id, min_cells=5, min_counts=500, remove_isotype=True)
    adata = stan_wrap(adata)
    adata.write(path / ('adata_'+sample_id+'.h5ad'))

Image weight: 0.0
Time elapsed: 38.46 seconds
{'lam1': 100.0, 'lam2': 1.0}
Spot-wise correlation:0.3139
Image weight: 0.1
Time elapsed: 37.49 seconds
{'lam1': 100.0, 'lam2': 1.0}
Spot-wise correlation:0.314
Image weight: 0.2
Time elapsed: 37.58 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.3138
Image weight: 0.30000000000000004
Time elapsed: 37.96 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.3136
Image weight: 0.4
Time elapsed: 37.87 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.3135
Image weight: 0.5
Time elapsed: 37.99 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.3132
Image weight: 0.6000000000000001
Time elapsed: 37.22 seconds
{'lam1': 0.01, 'lam2': 0.01}
Spot-wise correlation:0.3104
Image weight: 0.7000000000000001
Time elapsed: 37.18 seconds
{'lam1': 0.01, 'lam2': 0.01}
Spot-wise correlation:0.3104
Image weight: 0.8
Time elapsed: 37.74 seconds
{'lam1': 0.01, 'lam2': 0.01}
Spot-wise correlation:0.3104
Image weight: 0

### Glioblastoma

In [13]:
sample_id = "Parent_Visium_Human_Glioblastoma"
adata = stan.read_visium_sge(sample_id=sample_id, min_cells=5, min_counts=500)
adata = stan_wrap(adata)
adata.write(path / ('adata_'+sample_id+'.h5ad'))

Image weight: 0.0
Time elapsed: 22.83 seconds
{'lam1': 100.0, 'lam2': 1.0}
Spot-wise correlation:0.2378
Image weight: 0.1
Time elapsed: 21.75 seconds
{'lam1': 100.0, 'lam2': 1.0}
Spot-wise correlation:0.238
Image weight: 0.2
Time elapsed: 21.75 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2378
Image weight: 0.30000000000000004
Time elapsed: 22.13 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2376
Image weight: 0.4
Time elapsed: 21.94 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2372
Image weight: 0.5
Time elapsed: 21.86 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2368
Image weight: 0.6000000000000001
Time elapsed: 22.07 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2358
Image weight: 0.7000000000000001
Time elapsed: 21.90 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2346
Image weight: 0.8
Time elapsed: 22.05 seconds
{'lam1': 1.0, 'lam2': 0.01}
Spot-wise correlation:0.2325
Image weight: 

### Breast

In [11]:
sample_list = ["1142243F", "1160920F", "CID4290", "CID4535", "CID4465", "CID44971"]
for sample_id in sample_list:
    adata = stan.read_breast_wu("data/Breast_Wu/{}.h5ad".format(sample_id))
    adata = stan_wrap(adata)
    adata.write(path / ('adata_Breast_Wu_'+sample_id+'.h5ad'))

Image weight: 0.0
Time elapsed: 11.31 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2475
Image weight: 0.1
Time elapsed: 10.96 seconds
{'lam1': 100.0, 'lam2': 100.0}
Spot-wise correlation:0.2466
Image weight: 0.2
Time elapsed: 11.43 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2465
Image weight: 0.30000000000000004
Time elapsed: 11.06 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2462
Image weight: 0.4
Time elapsed: 10.83 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2458
Image weight: 0.5
Time elapsed: 10.93 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2449
Image weight: 0.6000000000000001
Time elapsed: 10.88 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2438
Image weight: 0.7000000000000001
Time elapsed: 10.89 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2425
Image weight: 0.8
Time elapsed: 10.82 seconds
{'lam1': 100.0, 'lam2': 0.01}
Spot-wise correlation:0.2409
Image w